In [ ]:
from langgraph.graph import StateGraph , START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv 
import os

In [ ]:
load_dotenv()  # take environment variables from .env

llm = ChatOpenAI(
    model="gpt-4o-mini",                # ✅ just the plain name
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1"
)

In [ ]:
class BlogState(TypedDict):
    topic: str
    outline: str
    blog_content : str
    evaluation : str

In [46]:
def genetare_outline(state: BlogState) -> BlogState:

    # extract the topic from state
    topic = state['topic']

    # call llm and generate outline
    prompt = f"Generate a detailed outline for a blog post about {topic}."

    # get the response from llm
    outline = llm.invoke(prompt).content

    # update the state with the outline
    state['outline'] = outline

    return state


In [ ]:
def create_blog_content(state: BlogState) -> BlogState:

    # extract the outline from state , joo bhi chahiye hoga hrr cheez ko lana pdega hrr function me 
    topic = state['topic']
    outline = state['outline']

    # call llm and generate blog content
    prompt = f"Write a detailed blog post based on the - {topic} following outline:\n{outline}"

    # get the response from llm
    blog_content = llm.invoke(prompt).content

    # update the state with the blog content
    state['blog_content'] = blog_content

    return state

In [ ]:
def evaluation(state: BlogState) -> BlogState:
    blog_content = state['blog_content']

    prompt = f"Evaluate the following blog content for clarity, coherence, and engagement:\n{blog_content}\nProvide constructive feedback and suggestions for improvement."

    evaluation = llm.invoke(prompt).content

    state['evaluation'] = evaluation

    return state

In [ ]:
# create graph

graph = StateGraph(BlogState)

# add nodes
graph.add_node('genetare_outline', genetare_outline)
graph.add_node("create_blog_content", create_blog_content)
graph.add_node('evaluation' , evaluation)

#add edges
graph.add_edge(START, 'genetare_outline')
graph.add_edge('genetare_outline', 'create_blog_content')
graph.add_edge('create_blog_content', 'evaluation')
graph.add_edge('evaluation', END)

# compile
workflow = graph.compile()

In [ ]:
# execute the graph
initial_state = {'topic': "The impact of artificial intelligence on modern healthcare"}

final_state = workflow.invoke(initial_state)

print(final_state) # sabkuch print krdega dict me jo bhi hoga -> topic , outlineb , blog_content

In [ ]:
print(final_state['outline']) #aise outline add krne se humko sirf answer dikhega

In [ ]:
print(final_state['blog_content']) #aise blog_content add krne se humko sirf answer dikhega

In [ ]:
print(final_state['evaluation']) #aise asnwer add krne se humko sirf answer dikhega

In [ ]:
from IPython.display import Image
Image(workflow.get_graph().draw_mermaid_png())